In [ ]:
import numpy as np
import joblib  # for saving the model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import KernelPCA, PCA
from sklearn.neighbors import KNeighborsClassifier
import optuna
import optuna.visualization as vis

n = 9

X = np.load('Datasets/kryptonite-%s-X.npy' % (n))
y = np.load('Datasets/kryptonite-%s-y.npy' % (n))

X_train_total, X_test, y_train_total, y_test = train_test_split(X, y, test_size=0.1, random_state=42)  # 10% of all data for final model evaluation

# To hold the best model and accuracy score
best_model = None
best_accuracy = -1

def objective(trial):
    global best_model, best_scaler, best_accuracy, best_pca

    # Shuffle and split data
    X_train, X_val, y_train, y_val = train_test_split(X_train_total, y_train_total, test_size=0.8, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    pca = PCA(n_components=trial.suggest_int('n_components', 1, X_train_scaled.shape[1]))
    X_train_scaled = pca.fit_transform(X_train_scaled)
    X_val_scaled = pca.transform(X_val_scaled)

    # KNN Classifier parameters
    classifier = KNeighborsClassifier(
        n_neighbors=trial.suggest_int('n_neighbors', 1, 50),
        weights=trial.suggest_categorical('weights', ['uniform', 'distance']),
        n_jobs=-1
    )

    # Train the classifier
    classifier.fit(X_train_scaled, y_train)

    # Evaluate on the validation set
    y_val_pred = classifier.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)

    # Report validation accuracy for pruning
    trial.report(val_accuracy, step=0)

    # Check if the trial should be pruned
    if trial.should_prune():
        raise optuna.TrialPruned()

    # If this is the best model so far, save it
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = classifier  # Update the best model
        best_scaler = scaler
        best_pca = pca
        # Save the best model
        joblib.dump(best_model, 'best_knn_model.joblib')
        print(f"New best model saved with accuracy: {best_accuracy:.4f}")

    return val_accuracy

# Create a study object with a pruner
pruner = optuna.pruners.MedianPruner(n_startup_trials=10)
study = optuna.create_study(direction="maximize", pruner=pruner)

# Optimize the objective function
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)
# print("Best model saved as 'best_knn_model.joblib'.")

# Optional: Visualize optimization results
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_slice(study).show()

# Evaluate on the validation set
X_test_scaled = best_scaler.transform(X_test)
X_test_scaled = best_pca.transform(X_test_scaled)
y_test_pred = best_model.predict(X_test_scaled)
val_accuracy = accuracy_score(y_test, y_test_pred)


[I 2024-11-13 22:25:41,647] A new study created in memory with name: no-name-bf6dc35d-383d-42b2-b06c-1ad503a8a625
[I 2024-11-13 22:25:42,267] Trial 0 finished with value: 0.12654320987654322 and parameters: {'n_components': 8, 'n_neighbors': 30, 'weights': 'uniform'}. Best is trial 0 with value: 0.12654320987654322.


New best model saved with accuracy: 0.1265


[I 2024-11-13 22:25:42,682] Trial 1 finished with value: 0.5459876543209876 and parameters: {'n_components': 2, 'n_neighbors': 27, 'weights': 'uniform'}. Best is trial 1 with value: 0.5459876543209876.


New best model saved with accuracy: 0.5460


[I 2024-11-13 22:25:43,112] Trial 2 finished with value: 0.551466049382716 and parameters: {'n_components': 2, 'n_neighbors': 20, 'weights': 'uniform'}. Best is trial 2 with value: 0.551466049382716.
[I 2024-11-13 22:25:43,175] Trial 3 finished with value: 0.5074845679012345 and parameters: {'n_components': 1, 'n_neighbors': 33, 'weights': 'distance'}. Best is trial 2 with value: 0.551466049382716.
[I 2024-11-13 22:25:43,230] Trial 4 finished with value: 0.7086419753086419 and parameters: {'n_components': 3, 'n_neighbors': 4, 'weights': 'distance'}. Best is trial 4 with value: 0.7086419753086419.


...

New best model saved with accuracy: 0.9330
New best model saved with accuracy: 0.9370


[I 2024-11-13 22:25:51,163] Trial 34 finished with value: 0.9368827160493827 and parameters: {'n_components': 7, 'n_neighbors': 4, 'weights': 'distance'}. Best is trial 33 with value: 0.937037037037037.
[I 2024-11-13 22:25:51,364] Trial 35 finished with value: 0.9128858024691358 and parameters: {'n_components': 8, 'n_neighbors': 9, 'weights': 'distance'}. Best is trial 33 with value: 0.937037037037037.
[I 2024-11-13 22:25:51,841] Trial 36 pruned. 
[I 2024-11-13 22:25:51,946] Trial 37 finished with value: 0.9368827160493827 and parameters: {'n_components': 7, 'n_neighbors': 4, 'weights': 'distance'}. Best is trial 33 with value: 0.937037037037037.
[I 2024-11-13 22:25:52,061] Trial 38 finished with value: 0.9305555555555556 and parameters: {'n_components': 8, 'n_neighbors': 6, 'weights': 'distance'}. Best is trial 33 with value: 0.937037037037037.
[I 2024-11-13 22:25:52,388] Trial 39 pruned. 
[I 2024-11-13 22:25:52,444] Trial 40 pruned. 
[I 2024-11-13 22:25:52,598] Trial 41 finished with

New best model saved with accuracy: 0.9400


[I 2024-11-13 22:25:53,748] Trial 49 finished with value: 0.9398148148148148 and parameters: {'n_components': 9, 'n_neighbors': 3, 'weights': 'distance'}. Best is trial 47 with value: 0.9399691358024691.
[I 2024-11-13 22:25:54,068] Trial 50 pruned. 
[I 2024-11-13 22:25:54,148] Trial 51 finished with value: 0.9398148148148148 and parameters: {'n_components': 9, 'n_neighbors': 3, 'weights': 'distance'}. Best is trial 47 with value: 0.9399691358024691.
[I 2024-11-13 22:25:54,224] Trial 52 finished with value: 0.9398148148148148 and parameters: {'n_components': 9, 'n_neighbors': 3, 'weights': 'distance'}. Best is trial 47 with value: 0.9399691358024691.
[I 2024-11-13 22:25:54,284] Trial 53 finished with value: 0.9232253086419753 and parameters: {'n_components': 9, 'n_neighbors': 2, 'weights': 'distance'}. Best is trial 47 with value: 0.9399691358024691.
[I 2024-11-13 22:25:54,516] Trial 54 pruned. 
[I 2024-11-13 22:25:54,667] Trial 55 finished with value: 0.9238425925925926 and parameters:

Best hyperparameters: {'n_components': 8, 'n_neighbors': 3, 'weights': 'distance'}
Best value: 0.9399691358024691
Best model saved as 'best_knn_model.joblib'.
